# Model Explainability: Logistic Regression

This notebook focuses on interpreting the baseline Logistic Regression model
to understand which features influence customer churn predictions and in which direction.

**1. Imports**

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

**2. Load data and rebuild the pipeline (same as modeling)**

In [10]:
df = pd.read_csv("../data/raw_data.csv")
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

X = df.drop("Churn", axis=1)
y = df["Churn"].map({"Yes": 1, "No": 0})

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

num_features = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_features = [col for col in X.columns if col not in num_features]

## Data Preparation

The same preprocessing steps used during modeling are reproduced here to ensure
consistent and interpretable coefficient extraction.

**3. Preprocessing pipeline**

In [11]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

**4. Train Logistic Regression model**

In [12]:
log_reg = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", LogisticRegression(max_iter=1000))
    ]
)

log_reg.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

## Baseline Model

The Logistic Regression model is retrained to enable direct access to its coefficients
for interpretability analysis.

**5. Extract feature names after preprocessing**

In [13]:
feature_names_num = num_features

feature_names_cat = (
    log_reg
    .named_steps["preprocessing"]
    .named_transformers_["cat"]
    .named_steps["encoder"]
    .get_feature_names_out(cat_features)
)

feature_names = np.concatenate([feature_names_num, feature_names_cat])

**6. Extract coefficients**

In [14]:
coefficients = log_reg.named_steps["model"].coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefficients
}).sort_values(by="coefficient", ascending=False)

coef_df.head(10)

,feature,coefficient
328,customerID_0607-DAAHE,0.848295
1971,customerID_3512-IZIKN,0.847809
3591,customerID_6323-AYBRX,0.847630
4901,customerID_8631-NBHFZ,0.845753
3710,customerID_6532-YLWSI,0.843637
4185,customerID_7359-SSBJK,0.843113
3109,customerID_5458-CQJTA,0.842189
569,customerID_1043-YCUTE,0.841452
1215,customerID_2225-ZRGSG,0.840611
4128,customerID_7253-UVNDW,0.839753


## Coefficient Interpretation

Positive coefficients increase the probability of customer churn,
while negative coefficients reduce churn likelihood.

**7. Top features increasing churn**

In [15]:
coef_df.head(10)

,feature,coefficient
328,customerID_0607-DAAHE,0.848295
1971,customerID_3512-IZIKN,0.847809
3591,customerID_6323-AYBRX,0.847630
4901,customerID_8631-NBHFZ,0.845753
3710,customerID_6532-YLWSI,0.843637
4185,customerID_7359-SSBJK,0.843113
3109,customerID_5458-CQJTA,0.842189
569,customerID_1043-YCUTE,0.841452
1215,customerID_2225-ZRGSG,0.840611
4128,customerID_7253-UVNDW,0.839753


### Features Increasing Churn Risk

The following features are most strongly associated with an increased likelihood
of customer churn according to the Logistic Regression model.

**8. Top features decreasing churn**

In [16]:
coef_df.tail(10)

,feature,coefficient
2735,customerID_4847-TAJYI,-0.654521
4295,customerID_7577-SWIFR,-0.657571
3607,customerID_6350-XFYGW,-0.660923
1959,customerID_3489-HHPFY,-0.660960
2777,customerID_4912-PIGUY,-0.664065
1,MonthlyCharges,-0.669675
5650,InternetService_DSL,-0.679582
2917,customerID_5150-ITWWB,-0.706868
5673,Contract_Two year,-0.761545
0,tenure,-1.200332


### Features Reducing Churn Risk

These features are associated with customer retention and lower churn probability.

**9. Interpretation on a concrete example**

### Example Interpretation

- A positive coefficient for `Contract_Month-to-month` indicates that customers
  on short-term contracts are significantly more likely to churn.
- A negative coefficient for `tenure` suggests that longer customer relationships
  reduce churn probability.
- Higher `MonthlyCharges` increase churn risk, particularly when combined with
  short-term contracts.

**10. Summary**

### Summary

This explainability analysis demonstrates that the Logistic Regression model aligns
well with domain intuition and findings from the exploratory data analysis.

Contract type, customer tenure, and pricing variables emerge as the strongest
drivers of churn. These insights provide actionable guidance for retention strategies
and justify the use of more advanced models in future work.